In [2]:
import re
import json
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
model_name = 'en_dense_lm_1_3b'
filenames = [*glob(f'../artifacts/results/{model_name}/*.json')]

In [4]:
data = []
for filename in filenames:
    # extract parameters from the filename using regex
    task_name, per_layer, k, seed, perm, lr, bsz, clip_norm = re.findall(
        r'(\w+)_per_layer_(0|1)_k_(\d+)_seed_(\d+)_perm_(\d+)_lr_([\d\.]+)_bsz_(\d+)_clip_norm_(\d+)', 
        filename.split('/')[-1])[0]
    with open(filename, 'r') as fp:
        cur_data = json.load(fp)
    cur_data.update({'task': task_name, 'method': ('Causal GD' if per_layer == '1' else 'GD'), 'k': k, 
                     'seed': seed, 'perm': perm, 'lr': lr, 'bsz': bsz, 
                     'clip_norm': clip_norm})
    data.append(cur_data)
data = pd.DataFrame.from_records(data)

In [5]:
metric_cols = ['SimAOU', 'Random SimAOU', 'ICL-FTZS SimAM']
agg_table = (data.groupby(['task', 'method', 'seed'])[metric_cols]
 .agg(list).applymap(lambda x: np.mean(x, axis=1).squeeze()))

In [6]:
agg_table.groupby(['task', 'method']).agg(['mean', 'std']).round(3)

SimAOU        Random SimAOU        ICL-FTZS SimAM       
                   mean    std          mean    std           mean    std
task   method                                                            
agnews Causal GD  0.327  0.007         0.001  0.001          0.432  0.015
       GD         0.308  0.039         0.001  0.001          0.505  0.052
cb     Causal GD  0.344  0.003         0.004  0.000          0.382  0.010
       GD         0.205  0.008         0.004  0.001          0.246  0.010
mr     Causal GD  0.328  0.007         0.001  0.000          0.263  0.005
       GD         0.216  0.009         0.001  0.000          0.139  0.053
sst2   Causal GD  0.223  0.010         0.002  0.000          0.299  0.018
       GD         0.111  0.002         0.002  0.000          0.150  0.021
sst5   Causal GD  0.107  0.005         0.002  0.000          0.326  0.006
       GD         0.092  0.013         0.002  0.000          0.308  0.018
subj   Causal GD  0.345  0.007         0.002  0.000          0.322  0.006
       GD         0.177  0.018         0.002  0.000          0.254  0.066